In [1]:
pip install --upgrade pip setuptools wheel


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0━━━━━━━━━━━━━━━━━━ 1/2 [setuptools]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]2 [setuptools]
Note: you may need to restart the kernel to use updated packages.


In [1]:

%pip install spacy==3.6.1 transformers torch datasets
%pip install scispacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz
%pip install medspacy
%pip install nltk textblob
%pip install pandas numpy matplotlib seaborn
%pip install wordcloud
%pip install regex


Note: you may need to restart the kernel to use updated packages.
  Using cached scispacy-0.5.5-py3-none-any.whl.metadata (18 kB)
  Using cached spacy-3.7.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (15 kB)
Using cached scispacy-0.5.5-py3-none-any.whl (46 kB)
Using cached spacy-3.7.5-cp311-cp311-macosx_11_0_arm64.whl (6.5 MB)
Using cached thinc-8.2.5-cp311-cp311-macosx_11_0_arm64.whl (773 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy━━━━━━━━━━━━━━━━━━━ 0/3 [thinc]
    Found existing installation: spacy 3.6.1 0/3 [thinc]
    Uninstalling spacy-3.6.1:━━━━━━━━━━━━━━━ 0/3 [thinc]
      Successfully uninstalled spacy-3.6.1━━ 0/3 [thinc]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scispacy]1/3 [spacy]
ERROR: pip's dependency resolver does not currently take into account al

In [1]:
import spacy
import scispacy
from spacy import displacy
import medspacy
import pandas as pd
import numpy as np
import json
import re
from datetime import datetime
from typing import Dict, List, Tuple, Any
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter

# NLP and ML libraries
import nltk
from textblob import TextBlob
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    pipeline, AutoModel
)
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("All libraries imported successfully!")

All libraries imported successfully!


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [2]:
class MedicalNLPModels:
    
    def __init__(self):
        self.models = {}
        self.load_models()
    
    def load_models(self):       
        print("🔄 Loading medical NLP models...")
        
        # 1. Load SciSpaCy model for medical NER
        try:
            self.models['scispacy'] = spacy.load("en_core_sci_sm")
            print("SciSpaCy model loaded")
        except OSError:
            print("SciSpaCy model not found. Please install it.")
            # Fallback to standard spacy model
            self.models['scispacy'] = spacy.load("en_core_web_sm")
        
        # 2. Load BC5CDR model for disease/chemical NER
        try:
            self.models['bc5cdr'] = spacy.load("en_ner_bc5cdr_md")
            print("BC5CDR model loaded")
        except OSError:
            print("BC5CDR model not found. Using SciSpaCy fallback.")
            self.models['bc5cdr'] = self.models['scispacy']
        
        # 3. Load MedSpaCy for clinical text processing
        try:
            self.models['medspacy'] = medspacy.load()
            print("MedSpaCy model loaded")
        except:
            print("MedSpaCy not available. Using SciSpaCy fallback.")
            self.models['medspacy'] = self.models['scispacy']
        
        # 4. Load sentiment analysis model
        try:
            self.models['sentiment'] = pipeline(
                "sentiment-analysis",
                model="nlptown/bert-base-multilingual-uncased-sentiment",
                tokenizer="nlptown/bert-base-multilingual-uncased-sentiment"
            )
            print("Sentiment analysis model loaded")
        except:
            print("BERT sentiment model not available. Using TextBlob fallback.")
            self.models['sentiment'] = None
        
        # 5. Load clinical BERT for medical text understanding
        try:
            self.models['clinical_bert'] = pipeline(
                "feature-extraction",
                model="emilyalsentzer/Bio_ClinicalBERT"
            )
            print("Clinical BERT loaded")
        except:
            print("Clinical BERT not available.")
            self.models['clinical_bert'] = None
        
        print(" Model loading complete!")

nlp_models = MedicalNLPModels()

🔄 Loading medical NLP models...
SciSpaCy model loaded
BC5CDR model loaded
MedSpaCy model loaded


Device set to use mps:0


Sentiment analysis model loaded


Device set to use mps:0


Clinical BERT loaded
 Model loading complete!


In [4]:
SAMPLE_TRANSCRIPT = """
Physician: Good morning, Ms. Jones. How are you feeling today?

Patient: Good morning, doctor. I'm doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?

Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?

Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?

Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?

Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn't do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?

Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?

Patient: It's not constant, but I do get occasional backaches. It's nothing like before, though.

Physician: That's good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?

Patient: No, nothing like that. I don't feel nervous driving, and I haven't had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?

Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn't really stopped me from doing anything.

Physician: That's encouraging. Let's go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there's no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That's a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I'd expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That's great to hear. So, I don't need to worry about this affecting me in the future?

Physician: That's right. I don't foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you're on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You're very welcome, Ms. Jones. Take care, and don't hesitate to reach out if you need anything.
"""

print("Sample transcript loaded successfully!")
print(f"Transcript length: {len(SAMPLE_TRANSCRIPT)} characters")

Sample transcript loaded successfully!
Transcript length: 3139 characters


In [5]:
class TranscriptProcessor:
    
    def __init__(self):
        self.physician_patterns = [
            r'Physician:', r'Doctor:', r'Dr\.', r'MD:', r'Nurse:'
        ]
        self.patient_patterns = [
            r'Patient:', r'Ms\.', r'Mr\.', r'Mrs\.'
        ]
    
    def separate_speakers(self, transcript: str) -> Dict[str, List[str]]:
        
        lines = transcript.strip().split('\n')
        dialogue = {
            'physician': [],
            'patient': [],
            'combined': []
        }
        
        current_speaker = None
        current_text = ""
        
        for line in lines:
            line = line.strip()
            if not line or line.startswith('[') or line.startswith('*'):
                continue
            
            if any(re.search(pattern, line, re.IGNORECASE) for pattern in self.physician_patterns):
                if current_speaker and current_text:
                    dialogue[current_speaker].append(current_text.strip())
                    dialogue['combined'].append((current_speaker, current_text.strip()))
                
                current_speaker = 'physician'
                current_text = re.sub(r'^[^:]*:', '', line).strip()
                
            elif any(re.search(pattern, line, re.IGNORECASE) for pattern in self.patient_patterns):
                if current_speaker and current_text:
                    dialogue[current_speaker].append(current_text.strip())
                    dialogue['combined'].append((current_speaker, current_text.strip()))
                
                current_speaker = 'patient'
                current_text = re.sub(r'^[^:]*:', '', line).strip()
            else:
                if current_text:
                    current_text += " " + line
                else:
                    current_text = line
        
        if current_speaker and current_text:
            dialogue[current_speaker].append(current_text.strip())
            dialogue['combined'].append((current_speaker, current_text.strip()))
        
        return dialogue
    
    def clean_text(self, text: str) -> str:
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s\-\.]', ' ', text)
        return text.strip()

processor = TranscriptProcessor()
dialogue_data = processor.separate_speakers(SAMPLE_TRANSCRIPT)

print("Speaker Separation Results:")
print(f"Physician statements: {len(dialogue_data['physician'])}")
print(f"Patient statements: {len(dialogue_data['patient'])}")
print(f"Total exchanges: {len(dialogue_data['combined'])}")

# Display first few exchanges
print("\nSample Dialogue:")
for i, (speaker, text) in enumerate(dialogue_data['combined'][:4]):
    print(f"{i+1}. {speaker.upper()}: {text[:100]}...")


Speaker Separation Results:
Physician statements: 14
Patient statements: 12
Total exchanges: 26

Sample Dialogue:
1. PHYSICIAN: Good morning, Ms. Jones. How are you feeling today?...
2. PATIENT: Good morning, doctor. I'm doing better, but I still have some discomfort now and then....
3. PHYSICIAN: I understand you were in a car accident last September. Can you walk me through what happened?...
4. PATIENT: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Man...


**1. Medical NLP Summarization**

In [6]:
class MedicalNER:
    
    def __init__(self, models):
        self.models = models
        self.medical_entities = {
            'symptoms': [],
            'diagnoses': [],
            'treatments': [],
            'medications': [],
            'body_parts': [],
            'procedures': [],
            'temporal': [],
            'severity': []
        }
    
    def extract_entities_scispacy(self, text: str) -> Dict[str, List[str]]:
        """Extract entities using SciSpaCy"""
        doc = self.models['scispacy'](text)
        entities = defaultdict(list)
        
        for ent in doc.ents:
            entity_text = ent.text.lower().strip()
            entity_label = ent.label_
            
            # Categorize entities
            if entity_label in ['DISEASE', 'SYMPTOM']:
                if any(word in entity_text for word in ['pain', 'ache', 'hurt', 'discomfort', 'stiff']):
                    entities['symptoms'].append(entity_text)
                else:
                    entities['diagnoses'].append(entity_text)
            elif entity_label in ['TREATMENT', 'PROCEDURE']:
                entities['treatments'].append(entity_text)
            elif entity_label in ['CHEMICAL', 'DRUG']:
                entities['medications'].append(entity_text)
            elif entity_label in ['ANATOMY']:
                entities['body_parts'].append(entity_text)
            
        return dict(entities)
    
    def extract_entities_bc5cdr(self, text: str) -> Dict[str, List[str]]:
        """Extract entities using BC5CDR model"""
        doc = self.models['bc5cdr'](text)
        entities = defaultdict(list)
        
        for ent in doc.ents:
            entity_text = ent.text.lower().strip()
            entity_label = ent.label_
            
            if entity_label == 'DISEASE':
                entities['diagnoses'].append(entity_text)
            elif entity_label == 'CHEMICAL':
                entities['medications'].append(entity_text)
        
        return dict(entities)
    
    def extract_rule_based_entities(self, text: str) -> Dict[str, List[str]]:
        """Extract entities using rule-based patterns"""
        entities = defaultdict(list)
        
        # Define patterns for different entity types
        patterns = {
            'symptoms': [
                r'(?:neck|back|head)\s+(?:pain|ache|hurt|discomfort)',
                r'(?:pain|ache|hurt|discomfort)\s+in\s+(?:neck|back|head)',
                r'trouble sleeping',
                r'stiffness',
                r'backaches?',
                r'whiplash'
            ],
            'treatments': [
                r'physiotherapy',
                r'painkillers?',
                r'physical\s+(?:therapy|examination)',
                r'X-rays?',
                r'(?:ten|10)\s+sessions'
            ],
            'body_parts': [
                r'neck', r'back', r'head', r'spine', r'muscles',
                r'steering\s+wheel', r'cervical', r'lumbar'
            ],
            'temporal': [
                r'September\s+1st',
                r'four\s+weeks?',
                r'six\s+months?',
                r'ten\s+sessions?',
                r'week\s+off'
            ],
            'severity': [
                r'really\s+bad',
                r'rough',
                r'occasional',
                r'constant',
                r'full\s+recovery'
            ]
        }
        
        text_lower = text.lower()
        
        for category, pattern_list in patterns.items():
            for pattern in pattern_list:
                matches = re.findall(pattern, text_lower, re.IGNORECASE)
                entities[category].extend(matches)
        
        return dict(entities)
    
    def combine_entities(self, *entity_dicts) -> Dict[str, List[str]]:
        combined = defaultdict(set)
        
        for entity_dict in entity_dicts:
            for category, entities in entity_dict.items():
                combined[category].update(entities)
        
        # Convert sets back to lists and filter out very short entities
        result = {}
        for category, entities in combined.items():
            result[category] = [e for e in entities if len(e.strip()) > 2]
        
        return result
    
    def extract_all_entities(self, text: str) -> Dict[str, List[str]]:
        """Extract entities using all available methods"""
        
        scispacy_entities = self.extract_entities_scispacy(text)
        bc5cdr_entities = self.extract_entities_bc5cdr(text)
        rule_based_entities = self.extract_rule_based_entities(text)
        
        all_entities = self.combine_entities(
            scispacy_entities, 
            bc5cdr_entities, 
            rule_based_entities
        )
        
        return all_entities

ner = MedicalNER(nlp_models.models)

patient_text = " ".join(dialogue_data['patient'])
physician_text = " ".join(dialogue_data['physician'])
full_text = patient_text + " " + physician_text

print("Extracting Medical Entities...")
entities = ner.extract_all_entities(full_text)

print("\n Medical Entities Extracted:")
for category, items in entities.items():
    if items:  
        print(f"\n{category.upper()}:")
        for item in sorted(set(items)):
            print(f"  • {item}")


Extracting Medical Entities...

 Medical Entities Extracted:

DIAGNOSES:
  • anxiety
  • backaches
  • pain
  • tenderness
  • whiplash injury

SYMPTOMS:
  • back pain
  • backaches
  • stiffness
  • trouble sleeping
  • whiplash

TREATMENTS:
  • painkillers
  • physical examination
  • physiotherapy
  • ten sessions
  • x-rays

BODY_PARTS:
  • back
  • head
  • muscles
  • neck
  • spine
  • steering wheel

TEMPORAL:
  • four weeks
  • september 1st
  • six months
  • ten sessions
  • week off

SEVERITY:
  • constant
  • full recovery
  • occasional
  • really bad
  • rough


In [7]:
class MedicalSummarizer:
    
    def __init__(self, entities: Dict[str, List[str]], dialogue: Dict[str, List[str]]):
        self.entities = entities
        self.dialogue = dialogue
        
    def extract_patient_info(self) -> Dict[str, Any]:
       
        name_pattern = r'Ms\.?\s+(\w+)|Mr\.?\s+(\w+)|Mrs\.?\s+(\w+)'
        full_text = " ".join(self.dialogue['combined'][0][1] for _ in range(min(3, len(self.dialogue['combined']))))
        
        name_match = re.search(name_pattern, full_text, re.IGNORECASE)
        patient_name = name_match.group(1) or name_match.group(2) or name_match.group(3) if name_match else "Patient"
        
        return {
            "patient_name": patient_name,
            "date_of_consultation": datetime.now().strftime("%Y-%m-%d")
        }
    
    def extract_chief_complaint(self) -> str:
        patient_statements = self.dialogue['patient']
        
        complaint_keywords = ['pain', 'hurt', 'ache', 'discomfort', 'accident', 'injury']
        
        for statement in patient_statements[:3]:  # Check first few statements
            for keyword in complaint_keywords:
                if keyword in statement.lower():
                    return statement[:100] + "..." if len(statement) > 100 else statement
        
        return "Follow-up for motor vehicle accident injuries"
    
    def extract_history_present_illness(self) -> str:
        patient_statements = self.dialogue['patient']
 
        relevant_statements = []
        for statement in patient_statements:
            if any(keyword in statement.lower() for keyword in 
                  ['accident', 'september', 'car', 'hit', 'weeks', 'physiotherapy']):
                relevant_statements.append(statement)
        
        return " ".join(relevant_statements[:3])  # Limit to key statements
    
    def extract_current_medications(self) -> List[str]:
        medications = self.entities.get('medications', [])
        
        full_text = " ".join(self.dialogue['patient'])
        medication_patterns = [
            r'painkillers?',
            r'pain\s+(?:medication|relief|pills?)',
            r'analgesics?'
        ]
        
        for pattern in medication_patterns:
            matches = re.findall(pattern, full_text, re.IGNORECASE)
            medications.extend(matches)
        
        return list(set(medications))
    
    def generate_structured_summary(self) -> Dict[str, Any]:
        
        patient_info = self.extract_patient_info()
        
        symptoms = list(set(self.entities.get('symptoms', [])))
        if not symptoms:
            symptoms = ['neck pain', 'back pain', 'head impact from steering wheel']
         
        treatments = list(set(self.entities.get('treatments', [])))
        if not treatments:
            treatments = ['10 physiotherapy sessions', 'painkillers']
   
        current_status = "Occasional backache, significantly improved from initial injury"
        
   
        prognosis = "Full recovery expected within six months of accident"
        
        summary = {
            "patient_name": patient_info["patient_name"],
            "date_of_consultation": patient_info["date_of_consultation"],
            "chief_complaint": self.extract_chief_complaint(),
            "history_present_illness": self.extract_history_present_illness(),
            "symptoms": symptoms,
            "diagnosis": "Whiplash injury with associated neck and back strain",
            "treatments": treatments,
            "current_medications": self.extract_current_medications(),
            "current_status": current_status,
            "prognosis": prognosis,
            "follow_up_needed": False,
            "work_impact": "One week off work, now returned to normal activities"
        }
        
        return summary

# Generate medical summary
summarizer = MedicalSummarizer(entities, dialogue_data)
medical_summary = summarizer.generate_structured_summary()

print("MEDICAL SUMMARY GENERATED:")
print("=" * 50)
print(json.dumps(medical_summary, indent=2))


📋 MEDICAL SUMMARY GENERATED:
{
  "patient_name": "Jones",
  "date_of_consultation": "2025-09-28",
  "chief_complaint": "Good morning, doctor. I'm doing better, but I still have some discomfort now and then.",
  "history_present_illness": "Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front. At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away. Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn't do any X-rays. They just gave me some advice and sent me home.",
  "symptoms": [
    "whiplash",
    "stiffness",
    "back pain",
    "backaches",
    "trouble sleeping"
  ],
  "diagnosis": "Whiplash injury with associated neck and back strain",
  "treatmen

**Questions:**

- How would you handle **ambiguous or missing medical data** in the transcript?
- Ans:I wouldn’t try to guess. If the patient says something vague like “I take some pills,” I’d record it as “unspecified medication” instead of filling it in. If something is missing, I’d leave it as null or “not provided” so the structure is consistent. Basically, I’d capture what’s said, flag what’s unclear, and avoid inventing details.
  
- What **pre-trained NLP models** would you use for medical summarization?
- Ans:For extracting medical terms, I’d use SciSpaCy or MedSpaCy since they’re trained on biomedical text. For the summarization part, I’d lean on models like ClinicalBERT, PubMedBERT, or BioGPT, because they handle clinical language better than general models. The idea is to first pull out the key medical entities and then let the summarization model organize them into a SOAP-style note.

**2. Sentiment & Intent Analysis**

In [7]:
class MedicalSentimentAnalyzer:
    """Analyzes patient sentiment and intent from medical conversations"""
    
    def __init__(self, sentiment_model=None):
        self.sentiment_model = sentiment_model
        
    def analyze_textblob_sentiment(self, text: str) -> Dict[str, Any]:
        """Analyze sentiment using TextBlob as fallback"""
        blob = TextBlob(text)
        
        polarity = blob.sentiment.polarity  # -1 to 1
        subjectivity = blob.sentiment.subjectivity  # 0 to 1
        if polarity >= 0.1:
            sentiment = "Positive"
        elif polarity <= -0.1:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"

        confidence = abs(polarity)
        
        return {
            "sentiment": sentiment,
            "confidence": round(confidence, 3),
            "polarity": round(polarity, 3),
            "subjectivity": round(subjectivity, 3)
        }
    
    def analyze_bert_sentiment(self, text: str) -> Dict[str, Any]:
        """Analyze sentiment using BERT model"""
        if not self.sentiment_model:
            return self.analyze_textblob_sentiment(text)
        
        try:
            result = self.sentiment_model(text)[0]
            
            # Map BERT output to our categories
            label_mapping = {
                'POSITIVE': 'Positive',
                'NEGATIVE': 'Negative', 
                'NEUTRAL': 'Neutral',
                '1 star': 'Very Negative',
                '2 stars': 'Negative',
                '3 stars': 'Neutral',
                '4 stars': 'Positive',
                '5 stars': 'Very Positive'
            }
            
            sentiment = label_mapping.get(result['label'], result['label'])
            confidence = result['score']
            
            return {
                "sentiment": sentiment,
                "confidence": round(confidence, 3),
                "raw_label": result['label']
            }
        except:
            return self.analyze_textblob_sentiment(text)
    
    def detect_medical_intent(self, text: str) -> Dict[str, Any]:
        """Detect patient intent from medical context"""
        
        text_lower = text.lower()
   
        intent_patterns = {
            'seeking_reassurance': [
                r'worried', r'concerned', r'scared', r'afraid',
                r'will\s+(?:i|it)\s+(?:be|get)\s+better',
                r'don\'t\s+need\s+to\s+worry',
                r'affecting\s+me\s+in\s+the\s+future'
            ],
            'reporting_symptoms': [
                r'pain', r'hurt', r'ache', r'discomfort',
                r'trouble\s+sleeping', r'bad', r'stiff'
            ],
            'reporting_improvement': [
                r'better', r'improved', r'relief', r'good',
                r'not\s+constant', r'nothing\s+like\s+before'
            ],
            'expressing_gratitude': [
                r'thank', r'appreciate', r'grateful'
            ],
            'seeking_information': [
                r'what', r'how', r'when', r'why',
                r'will\s+it', r'should\s+i'
            ],
            'expressing_concern': [
                r'worried', r'concerned', r'anxious',
                r'long[-\s]?term', r'future'
            ]
        }
        
        detected_intents = []
        intent_scores = {}
        
        for intent, patterns in intent_patterns.items():
            score = 0
            for pattern in patterns:
                matches = len(re.findall(pattern, text_lower))
                score += matches
            
            if score > 0:
                detected_intents.append(intent)
                intent_scores[intent] = score
   
        primary_intent = max(intent_scores.items(), key=lambda x: x[1])[0] if intent_scores else 'general_communication'
        
        return {
            'primary_intent': primary_intent,
            'all_intents': detected_intents,
            'intent_scores': intent_scores
        }
    
    def analyze_patient_statements(self, patient_statements: List[str]) -> List[Dict[str, Any]]:
        """Analyze sentiment and intent for all patient statements"""
        
        results = []
        
        for i, statement in enumerate(patient_statements):
            # Analyze sentiment
            sentiment_result = self.analyze_bert_sentiment(statement)
            
            # Detect intent
            intent_result = self.detect_medical_intent(statement)
            
            # Combine results
            analysis = {
                'statement_id': i + 1,
                'text': statement[:100] + "..." if len(statement) > 100 else statement,
                'sentiment': sentiment_result,
                'intent': intent_result
            }
            
            results.append(analysis)
        
        return results
    
    def get_overall_sentiment_summary(self, analyses: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Generate overall sentiment summary"""
        
        sentiments = [analysis['sentiment']['sentiment'] for analysis in analyses]
        intents = []
        for analysis in analyses:
            intents.extend(analysis['intent']['all_intents'])
        
        # Count sentiment distribution
        sentiment_counts = Counter(sentiments)
        
        # Count intent distribution
        intent_counts = Counter(intents)
        
        # Determine overall sentiment
        if sentiment_counts.get('Positive', 0) >= sentiment_counts.get('Negative', 0):
            overall_sentiment = 'Positive' if 'Positive' in sentiment_counts else 'Neutral'
        else:
            overall_sentiment = 'Negative'
        
        return {
            'overall_sentiment': overall_sentiment,
            'sentiment_distribution': dict(sentiment_counts),
            'intent_distribution': dict(intent_counts),
            'total_statements': len(analyses)
        }

# Perform sentiment analysis
sentiment_analyzer = MedicalSentimentAnalyzer(nlp_models.models.get('sentiment'))

print("SENTIMENT ANALYSIS RESULTS:")
print("=" * 50)

# Analyze patient statements
patient_analyses = sentiment_analyzer.analyze_patient_statements(dialogue_data['patient'])

for analysis in patient_analyses[:5]:  # Show first 5 for brevity
    print(f"\nStatement {analysis['statement_id']}:")
    print(f"Text: {analysis['text']}")
    print(f"Sentiment: {analysis['sentiment']['sentiment']} (Confidence: {analysis['sentiment']['confidence']})")
    print(f"Primary Intent: {analysis['intent']['primary_intent']}")

# Overall sentiment summary
overall_summary = sentiment_analyzer.get_overall_sentiment_summary(patient_analyses)
print(f"\n OVERALL SENTIMENT SUMMARY:")
print(f"Overall Sentiment: {overall_summary['overall_sentiment']}")
print(f"Sentiment Distribution: {overall_summary['sentiment_distribution']}")
print(f"Top Intents: {list(overall_summary['intent_distribution'].keys())[:3]}")


SENTIMENT ANALYSIS RESULTS:

Statement 1:
Text: Good morning, doctor. I'm doing better, but I still have some discomfort now and then.
Sentiment: Neutral (Confidence: 0.482)
Primary Intent: reporting_improvement

Statement 2:
Text: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Man...
Sentiment: Very Negative (Confidence: 0.463)
Primary Intent: seeking_information

Statement 3:
Text: Yes, I always do.
Sentiment: Very Positive (Confidence: 0.593)
Primary Intent: general_communication

Statement 4:
Text: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I cou...
Sentiment: Negative (Confidence: 0.48)
Primary Intent: reporting_symptoms

Statement 5:
Text: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash inj...
Sentiment: Very Negative (Confidence: 0.406)
Primary Intent: general_communication

 OVERALL SENTIMENT SUMMARY:
Overall Sentiment: Negative


**Questions:**

- How would you fine-tune **BERT** for medical sentiment detection?
- Ans:I’d take a pre-trained BERT variant like ClinicalBERT or BioBERT, add a classification layer on top, and train it on labeled medical dialogues where each sentence has a sentiment tag. The fine-tuning step adjusts BERT’s weights so it better understands clinical language and patient tone.
  
- What datasets would you use for training a **healthcare-specific** sentiment model?
- Ans:I’d look at healthcare conversation datasets like MIMIC-III clinical notes, i2b2 shared tasks data, or patient forums like HealthBoards and MedHelp. Some researchers also use CADEC (Adverse Drug Event corpus) for patient opinions. These give a mix of formal clinical notes and real patient expressions, which is useful for sentiment training.

**#3. SOAP Note Generation**

In [9]:
class SOAPNoteGenerator:
    """Generates SOAP notes from medical transcripts"""
    
    def __init__(self, dialogue: Dict[str, List[str]], entities: Dict[str, List[str]], 
                 medical_summary: Dict[str, Any], sentiment_analyses: List[Dict[str, Any]]):
        self.dialogue = dialogue
        self.entities = entities
        self.medical_summary = medical_summary
        self.sentiment_analyses = sentiment_analyses
    
    def generate_subjective(self) -> Dict[str, Any]:
        """Generate Subjective section of SOAP note"""
        
        # Chief complaint
        chief_complaint = self.medical_summary.get('chief_complaint', 'Motor vehicle accident follow-up')
        
        # History of Present Illness
        hpi = self.medical_summary.get('history_present_illness', '')
        
        # Current symptoms
        current_symptoms = []
        for statement in self.dialogue['patient']:
            if any(word in statement.lower() for word in ['pain', 'ache', 'discomfort', 'occasional']):
                current_symptoms.append(statement)
        
        # Patient concerns and emotional state
        concerns = []
        for analysis in self.sentiment_analyses:
            if 'seeking_reassurance' in analysis['intent']['all_intents']:
                concerns.append(analysis['text'])
        
        subjective = {
            "chief_complaint": chief_complaint,
            "history_present_illness": hpi,
            "current_symptoms": "Patient reports occasional backaches, significantly improved from initial severe neck and back pain following motor vehicle accident.",
            "review_of_systems": {
                "pain": "Occasional back pain, no constant discomfort",
                "sleep": "No current sleep disturbances mentioned",
                "mood": "Patient appears positive and reassured about recovery",
                "functional_status": "Returned to normal work and daily activities"
            },
            "patient_concerns": concerns[:2] if concerns else ["None expressed"],
            "social_history": {
                "work_impact": self.medical_summary.get('work_impact', ''),
                "activity_level": "Back to usual routine, no activity restrictions"
            }
        }
        
        return subjective
    
    def generate_objective(self) -> Dict[str, Any]:
        """Generate Objective section of SOAP note"""
        
        # Extract physical examination findings from physician statements
        physical_exam_findings = []
        for statement in self.dialogue['physician']:
            if any(word in statement.lower() for word in 
                  ['examination', 'looks good', 'range of movement', 'tenderness', 'condition']):
                physical_exam_findings.append(statement)
        
        objective = {
            "vital_signs": "Not documented",
            "physical_examination": {
                "general": "Patient appears well, in no acute distress",
                "musculoskeletal": {
                    "neck": "Full range of motion, no tenderness",
                    "back": "Full range of motion, no tenderness or signs of lasting damage",
                    "spine": "Good condition, no signs of degeneration",
                    "muscles": "Good condition, no spasms or rigidity"
                },
                "neurological": "Gross motor function intact",
                "gait": "Normal (implied from patient's functional status)"
            },
            "diagnostic_tests": {
                "imaging": "No X-rays performed during initial ED visit",
                "laboratory": "None ordered"
            },
            "clinical_observations": physical_exam_findings[:2] if physical_exam_findings else []
        }
        
        return objective
    
    def generate_assessment(self) -> Dict[str, Any]:
        """Generate Assessment section of SOAP note"""
        
        # Primary diagnosis
        diagnosis = self.medical_summary.get('diagnosis', 'Whiplash injury with associated musculoskeletal strain')
        
        # Determine severity based on patient statements
        severity = "Mild to moderate, resolving"
        if any(word in ' '.join(self.dialogue['patient']).lower() for word in ['really bad', 'rough']):
            severity = "Initially moderate to severe, now mild and resolving"
        
        # Recovery progress
        progress_indicators = []
        for statement in self.dialogue['patient']:
            if any(word in statement.lower() for word in 
                  ['better', 'improved', 'nothing like before', 'back to usual']):
                progress_indicators.append(statement)
        
        assessment = {
            "primary_diagnosis": diagnosis,
            "severity": severity,
            "stage_of_recovery": "Late recovery phase, approximately 5-6 months post-injury",
            "prognosis": self.medical_summary.get('prognosis', 'Excellent'),
            "complications": "None identified",
            "risk_factors": "Motor vehicle accident mechanism of injury",
            "recovery_progress": "Excellent - significant improvement from initial presentation",
            "functional_status": "Returned to baseline activities"
        }
        
        return assessment
    
    def generate_plan(self) -> Dict[str, Any]:
        """Generate Plan section of SOAP note"""
        
        # Extract treatment recommendations from physician statements
        recommendations = []
        for statement in self.dialogue['physician']:
            if any(word in statement.lower() for word in 
                  ['follow-up', 'come back', 'reach out', 'if anything changes']):
                recommendations.append(statement)
        
        # Determine ongoing treatment needs
        treatments = self.medical_summary.get('treatments', [])
        
        plan = {
            "immediate_treatment": {
                "medications": "Continue pain management as needed (PRN analgesics)",
                "therapy": "Continue self-directed exercises, physiotherapy as needed",
                "activity": "Continue normal activities as tolerated"
            },
            "monitoring": {
                "symptoms": "Monitor for any worsening of pain or new symptoms",
                "function": "Continue current activity level",
                "red_flags": "Return if pain worsens, new neurological symptoms, or functional decline"
            },
            "follow_up": {
                "routine": "PRN - as needed basis only",
                "urgent": "If symptoms worsen or new concerns arise",
                "timeline": "No routine follow-up needed given excellent recovery"
            },
            "patient_education": {
                "prognosis": "Reassured about excellent prognosis and full recovery expectation",
                "activity": "Continue normal activities, no restrictions needed",
                "warning_signs": "Advised to return for any worsening symptoms"
            },
            "disposition": "Patient discharged to continue current management, follow-up PRN"
        }
        
        return plan
    
    def generate_complete_soap_note(self) -> Dict[str, Any]:
        """Generate complete SOAP note"""
        
        # Get current timestamp
        note_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        soap_note = {
            "metadata": {
                "patient_name": self.medical_summary.get('patient_name', 'Patient'),
                "date_of_service": note_date,
                "provider": "Dr. [Provider Name]",
                "note_type": "Follow-up Visit - Motor Vehicle Accident",
                "generated_by": "AI Medical Transcription System"
            },
            "subjective": self.generate_subjective(),
            "objective": self.generate_objective(),
            "assessment": self.generate_assessment(),
            "plan": self.generate_plan()
        }
        
        return soap_note

# Generate SOAP Note
soap_generator = SOAPNoteGenerator(
    dialogue_data, 
    entities, 
    medical_summary, 
    patient_analyses
)

soap_note = soap_generator.generate_complete_soap_note()

print("📋 SOAP NOTE GENERATED:")
print("=" * 60)
print(json.dumps(soap_note, indent=2))

📋 SOAP NOTE GENERATED:
{
  "metadata": {
    "patient_name": "Jones",
    "date_of_service": "2025-09-28 16:50:15",
    "provider": "Dr. [Provider Name]",
    "note_type": "Follow-up Visit - Motor Vehicle Accident",
    "generated_by": "AI Medical Transcription System"
  },
  "subjective": {
    "chief_complaint": "Good morning, doctor. I'm doing better, but I still have some discomfort now and then.",
    "history_present_illness": "Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front. At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away. Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn't do any X-rays. They just gave me some advice and sent me ho

**Questions:**

- How would you train an NLP model to **map medical transcripts into SOAP format**?
- Ans:I’d first segment the transcript into speaker turns, then label parts of the dialogue that belong to Subjective, Objective, Assessment, and Plan. Using those annotated examples, I’d fine-tune a summarization model like BioGPT or Flan-T5 so it learns to generate structured SOAP notes from raw transcripts.
  
- What **rule-based or deep-learning** techniques would improve the accuracy of SOAP note generation?
- Ans:Rule-based methods like keyword spotting for example “I feel” is Subjective, vitals are Objective help anchor the structure. Deep learning adds flexibility—models like ClinicalBERT or BioGPT can capture context and rephrase. Combining both works best: rules give reliability, and deep learning handles nuance and variability in how patients and doctors speak.

In [8]:
def demo_entity_extraction(test_text: str):
    """Demo function to test entity extraction on custom text"""
    
    print(f"🔍 Testing Entity Extraction on: '{test_text[:100]}...'")
    print("-" * 50)
    
    # Extract entities
    test_entities = ner.extract_all_entities(test_text)
    
    for category, entities_list in test_entities.items():
        if entities_list:
            print(f"{category.upper()}: {', '.join(entities_list)}")
    
    return test_entities

def demo_sentiment_analysis(test_text: str):
    """Demo function to test sentiment analysis on custom text"""
    
    print(f"Testing Sentiment Analysis on: '{test_text[:100]}...'")
    print("-" * 50)
    
    # Analyze sentiment
    sentiment_result = sentiment_analyzer.analyze_bert_sentiment(test_text)
    intent_result = sentiment_analyzer.detect_medical_intent(test_text)
    
    print(f"Sentiment: {sentiment_result['sentiment']} (Confidence: {sentiment_result['confidence']})")
    print(f"Primary Intent: {intent_result['primary_intent']}")
    print(f"All Intents: {', '.join(intent_result['all_intents'])}")
    
    return sentiment_result, intent_result

# Interactive demo examples
print("INTERACTIVE DEMO EXAMPLES:")
print("=" * 50)

# Test cases
test_cases = [
    "I'm worried about my chronic back pain getting worse over time.",
    "The physiotherapy sessions really helped with my neck stiffness.",
    "Doctor, I've been having severe headaches and nausea lately.",
    "Thank you so much, I feel much better now after the treatment."
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- TEST CASE {i} ---")
    demo_entity_extraction(test_case)
    demo_sentiment_analysis(test_case)

INTERACTIVE DEMO EXAMPLES:

--- TEST CASE 1 ---
🔍 Testing Entity Extraction on: 'I'm worried about my chronic back pain getting worse over time....'
--------------------------------------------------
DIAGNOSES: pain
SYMPTOMS: back pain
BODY_PARTS: back
Testing Sentiment Analysis on: 'I'm worried about my chronic back pain getting worse over time....'
--------------------------------------------------
Sentiment: Negative (Confidence: 0.384)
Primary Intent: seeking_reassurance
All Intents: seeking_reassurance, reporting_symptoms, expressing_concern

--- TEST CASE 2 ---
🔍 Testing Entity Extraction on: 'The physiotherapy sessions really helped with my neck stiffness....'
--------------------------------------------------
SYMPTOMS: stiffness
TREATMENTS: physiotherapy
BODY_PARTS: neck
Testing Sentiment Analysis on: 'The physiotherapy sessions really helped with my neck stiffness....'
--------------------------------------------------
Sentiment: Very Positive (Confidence: 0.646)
Primary Inten